In [50]:
import matplotlib.pyplot as plt
import math
import time
import random
import copy

import numpy as np
import copy
import torch
import cv2

In [104]:
class Agent_Conv:
    def __init__(self, num_input, layers, num_output):
        assert type(layers)==type([]), "Error with layers, give array of the number of layers"
        self.num_input = num_input  #set input number
        self.num_output = num_output #set ooutput number
        self.hidden=[]
        last=num_input
        self.num_genes=0
        for layer in layers:
            self.hidden.append(layer)
            self.num_genes+=(last * layer)
            last=layer
        self.num_genes +=(self.hidden[-1]*num_output)+num_output
        self.weights = None
        self.hidden_weights=None
        self.bias = None
        print("Auto",self.num_genes)
    def set_genes(self, gene):
        weight_idxs = self.num_input * self.hidden[0] #size of weights to hidden
        current=weight_idxs
        weights_idxs=[current] #start with end of last
        for i in range(len(self.hidden)-1):
            current+=self.hidden[i]*self.hidden[i+1] #calculate next idx for each layer
            weights_idxs.append(current)
        bias_idxs=None
        weights_idxs.append(self.hidden[-1] * self.num_output + weights_idxs[-1]) #add last layer heading to output
        bias_idxs = weights_idxs[-1]+ self.num_output #sizes of biases
        w = gene[0 : weight_idxs].reshape(self.hidden[0], self.num_input)   #merge genes
        ws=[]
        for i in range(len(self.hidden)-1):
            ws.append(gene[weights_idxs[i] : weights_idxs[i+1]].reshape(self.hidden[i+1], self.hidden[i]))
        ws.append(gene[weights_idxs[-2] : weights_idxs[-1]].reshape(self.num_output, self.hidden[-1]))
        b = gene[weights_idxs[-1]: bias_idxs].reshape(self.num_output,) #merge genes

        self.weights = torch.from_numpy(w) #assign weights
        self.hidden_weights=[]
        for w in ws:
            self.hidden_weights.append(torch.from_numpy(w))
        self.bias = torch.from_numpy(b) #assign biases

    def forward(self, x):
        #create conv layer
        print(len(self.weights))
        m=torch.nn.Conv2d(1, 5, 1, stride=2, padding=3, bias=False)
        x=x[np.newaxis, :, :]
        x=x[:,:,:,np.newaxis]
        x=m(torch.tensor(x).float())
        H=len(x)
        x=x.flatten()
        print(x)
        print(x.shape,self.weights.T.shape)
        #x = torch.tensor(np.dot(self.weights.T.float(),x.detach().numpy()).flatten())
        #run through forward layers
        
        for i in range(len(self.hidden_weights)-1):
            x =torch.mm(x,self.hidden_weights[i].T.float()) #second layer
        return torch.mm(x,self.hidden_weights[-1].T.float()) + self.bias #third layer
    
    def get_action(self, x):
        vectors=[(1,1),(1,0),(0,1),(-1,-1),(-1,0),(0,-1),(-1,1),(1,-1)] #possible moves
        arr=list(self.forward(x)[0])
        ind=np.argmax(arr)
        return vectors[ind]

In [105]:
whegBot=Agent_Conv(5*5,[10,10],10) #define the agent


input_=np.random.rand(5,5)
output_=np.random.rand(10)

whegBot.set_genes(np.random.normal(0, 0.5, (whegBot.num_genes)))
print(input_)

whegBot.get_action(input_)

Auto 460
[[0.09071728 0.89048101 0.91653832 0.48596543 0.70704021]
 [0.90111468 0.94624615 0.50836628 0.73522498 0.26983969]
 [0.274328   0.65985784 0.51296762 0.36587234 0.39277505]
 [0.16635284 0.57445914 0.4890709  0.99624912 0.61081912]
 [0.73205754 0.94518278 0.36127143 0.3382349  0.18230279]]
10


RuntimeError: Given groups=1, weight of size [5, 1, 1, 1], expected input[1, 5, 5, 1] to have 1 channels, but got 5 channels instead